# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys
import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
# Import necessary libraries for text processing
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import warnings

# Ignore all warnings
warnings.simplefilter('ignore')
# download necessary data from nltk
nltk.download(['punkt_tab', 'stopwords', 'wordnet'])
np.random.seed(42)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
database_filepath = './data/DisasterResponse.db'

In [3]:
# load data from database
engine = create_engine(f'sqlite:///{database_filepath}')
df = pd.read_sql_table('disaster_messages', engine)
X = df['message']
Y = df.iloc[:, 4:]  # Assuming the first four columns are not target variables
category_names = Y.columns.tolist()

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    """
    Tokenize and process text data.

    Args:
        text (str): Input text to tokenize.

    Returns:
        list: List of processed tokens.
    """
    

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    
    clean_tokens = [lemmatizer.lemmatize(token).lower().strip() 
                    for token in tokens if token.lower() not in stop_words]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def evaluate_model(model, X_test, Y_test, category_names):
    """
    Evaluate the model's performance on test data.

    Args:
        model: Trained model.
        X_test (pd.Series): Test features.
        Y_test (pd.DataFrame): Test target variables.
        category_names (list): List of category names.

    Returns:
        None
    """
    Y_pred = model.predict(X_test)
    for i, column in enumerate(category_names):
        print(f"Category: {column}")
        print(classification_report(Y_test.iloc[:, i], Y_pred[:, i]))

In [8]:
evaluate_model(pipeline, X_test, Y_test, category_names)

Category: related
              precision    recall  f1-score   support

           0       0.72      0.34      0.46      1266
           1       0.81      0.96      0.88      3938
           2       0.60      0.07      0.13        40

    accuracy                           0.80      5244
   macro avg       0.71      0.46      0.49      5244
weighted avg       0.79      0.80      0.77      5244

Category: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.84      0.47      0.60       895

    accuracy                           0.89      5244
   macro avg       0.87      0.72      0.77      5244
weighted avg       0.89      0.89      0.88      5244

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.5

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
parameters = {
        'clf__estimator__n_estimators': [10, 50, 100],
        'clf__estimator__min_samples_split': [2, 4],
    }
    
cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, verbose=3)

In [10]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV 1/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10;, score=0.230 total time=  30.7s
[CV 2/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10;, score=0.227 total time=  30.8s
[CV 3/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10;, score=0.228 total time=  30.7s
[CV 1/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50;, score=0.243 total time=  52.2s
[CV 2/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50;, score=0.240 total time=  52.1s
[CV 3/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50;, score=0.256 total time=  52.2s
[CV 1/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100;, score=0.241 total time= 1.4min
[CV 2/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100;, score=0.242 total time= 1.4min
[CV 3/3] END clf__

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(token_pattern=None,
                                                        tokenizer=<function tokenize at 0x000001C454B6DFC0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))]),
             param_grid={'clf__estimator__min_samples_split': [2, 4],
                         'clf__estimator__n_estimators': [10, 50, 100]},
             verbose=3)

In [11]:
cv.best_params_

{'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 100}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
evaluate_model(cv, X_test, Y_test, category_names)


Category: related
              precision    recall  f1-score   support

           0       0.73      0.34      0.47      1266
           1       0.81      0.96      0.88      3938
           2       0.67      0.05      0.09        40

    accuracy                           0.80      5244
   macro avg       0.74      0.45      0.48      5244
weighted avg       0.79      0.80      0.77      5244

Category: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.84      0.48      0.61       895

    accuracy                           0.90      5244
   macro avg       0.87      0.73      0.78      5244
weighted avg       0.89      0.90      0.88      5244

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.5

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### Description of the Approach

This approach enhances the feature extraction process by combining multiple feature types using `FeatureUnion`. 

- **Text-based features**: Traditional features like TF-IDF are extracted using `CountVectorizer` and `TfidfTransformer`.
- **Custom features**: Additional features, such as text length, are incorporated using a custom transformer (`TextLengthExtractor`).

The combined features are then passed to a machine learning model (e.g., `RandomForestClassifier`) for training. This method allows the pipeline to capture richer information, improving the model's ability to generalize across diverse data.


In [19]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
import numpy as np

# Custom Transformer to add additional feature (example: document length)
class TextLengthExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Calculate the length of each document
        return np.array([[len(text)] for text in X])

# Pipeline with additional feature
pipeline_wc = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None)),
            ('tfidf', TfidfTransformer())
        ])),
        ('text_length', TextLengthExtractor())  # Adding custom feature
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
])



In [20]:
pipeline_wc.fit(X_train, Y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(token_pattern=None,
                                                                                  tokenizer=<function tokenize at 0x000001C454B6DFC0>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('text_length',
                                                 TextLengthExtractor())])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))])

In [22]:
evaluate_model(pipeline_wc, X_test, Y_test, category_names)

Category: related
              precision    recall  f1-score   support

           0       0.76      0.34      0.47      1242
           1       0.82      0.97      0.88      3959
           2       0.50      0.07      0.12        43

    accuracy                           0.81      5244
   macro avg       0.69      0.46      0.49      5244
weighted avg       0.80      0.81      0.78      5244

Category: request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      4384
           1       0.84      0.50      0.63       860

    accuracy                           0.90      5244
   macro avg       0.88      0.74      0.79      5244
weighted avg       0.90      0.90      0.89      5244

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5222
           1       0.00      0.00      0.00        22

    accuracy                           1.00      5244
   macro avg       0.5

### 9. Export your model as a pickle file

In [13]:
def save_model(model, model_filepath):
    """
    Save the trained model to a file.

    Args:
        model: Trained model.
        model_filepath (str): Filepath to save the model.

    Returns:
        None
    """
    with open(model_filepath, 'wb') as file:
        pickle.dump(model, file)

In [14]:
save_model(cv, './models/classifier.pkl')

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.